# EcoPlan Urbano – Análisis integrado
Este notebook reproduce la lógica del manual metodológico: descarga colecciones satelitales, calcula índices compuestos (vegetación, calor, calidad del aire, riesgo hídrico) e integra datos socioeconómicos para generar capas listas para paneles o reportes.

In [ ]:
import ee
import geemap
import datetime

ee.Initialize(project='github-nasa')

# Parámetros de ejecución
AOI = ee.Geometry.Polygon([[
START_DATE = '2024-01-01'
END_DATE = '2024-12-31'
CLOUD_PCT = 20

In [ ]:
def mask_sentinel2(image):
    scl = image.select('SCL')
    mask = scl.neq(3).And(scl.neq(7)).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)).And(scl.neq(11))
    return image.updateMask(mask)

sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
sentinel = (sentinel

ndvi = sentinel.map(lambda img: img.normalizedDifference(['B8', 'B4']).rename('NDVI'))
ndwi = sentinel.map(lambda img: img.normalizedDifference(['B3', 'B8']).rename('NDWI'))
ndbi = sentinel.map(lambda img: img.normalizedDifference(['B11', 'B8']).rename('NDBI'))

ndvi_max = ndvi.qualityMosaic('NDVI').clip(AOI)
ndwi_mean = ndwi.mean().clip(AOI)
impervious = ndbi.median().clip(AOI)

landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')

def add_lst(image):
    lst = image.select('ST_B10').multiply(0.00341802).add(149).subtract(273.15)
    return image.addBands(lst.rename('LST_C'))

lst = landsat.map(add_lst).median().select('LST_C').clip(AOI)

aod = ee.ImageCollection('MODIS/061/MOD08_M3')

no2 = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_NO2')

pm25 = ee.ImageCollection('NASA/SEDAC/SDG/SDG11_6_2/PM2_5')

srtm = ee.Image('USGS/SRTMGL1_003').clip(AOI)
slope = ee.Terrain.slope(srtm).rename('Slope')

green_mask = ndvi_max.select('NDVI').gt(0.4)
green_area = green_mask.multiply(ee.Image.pixelArea())
population = ee.ImageCollection('CIESIN/GPWv411/GPW_Population_Count')

pixel_population = population.max(ee.Image.constant(1))
green_per_capita = green_area.divide(pixel_population).rename('GreenPerCapita')

In [ ]:
Map = geemap.Map(center=[-12.05, -77.05], zoom=11)
Map.addLayer(ndvi_max.select('NDVI'), {'min': -0.2, 'max': 0.8, 'palette': ['#002651', '#22c55e']}, 'NDVI máximo')
Map.addLayer(lst, {'min': 20, 'max': 40, 'palette': ['#00b3ff', '#f97316', '#7f1d1d']}, 'Temperatura superficial (°C)')
Map.addLayer(aod, {'min': 0, 'max': 1.5, 'palette': ['#22c55e', '#facc15', '#b91c1c']}, 'AOD promedio')
Map.addLayer(no2, {'min': 0, 'max': 3e-4, 'palette': ['#0ea5e9', '#f97316', '#7f1d1d']}, 'NO₂ troposférico')
Map.addLayer(pm25, {'min': 0, 'max': 80, 'palette': ['#22c55e', '#f97316', '#7f1d1d']}, 'PM₂.₅ estimado')
Map.addLayer(green_per_capita, {'min': 0, 'max': 30, 'palette': ['#7f1d1d', '#facc15', '#22c55e']}, 'm² de áreas verdes por habitante')
Map.add_colorbar(label='Escala normalizada', cmap='RdYlGn', vmin=0, vmax=1)
Map